# 🟣 Brief 5 : Pipeline ETL complet

**Badge:** 🟣 Expert  
**Durée:** 3 heures  
**Sections validées:** 06-Data-Engineering + 04-Stdlib + 05-Qualité-Tests

---

## 📋 Contexte

Vous êtes **Data Engineer** dans une fintech. Votre équipe doit ingérer des données de plusieurs sources (fichiers historiques CSV + API REST) pour alimenter un data warehouse.

Le pipeline doit :
- Extraire les données depuis plusieurs sources
- Valider la qualité des données (schéma, types, contraintes)
- Transformer et enrichir les données
- Charger dans un data warehouse (DuckDB)
- Être robuste, testé, et respecter les bonnes pratiques de qualité

**Votre mission :** Construire un pipeline ETL professionnel avec validation Pydantic, tests, logging structuré, et configuration par environnement.

---

## 🎯 Objectifs du projet

Construire un pipeline ETL complet qui :

1. ✅ **Extract** : Lire CSV + appeler une API REST
2. ✅ **Transform** : Valider avec Pydantic + nettoyer avec Pandas
3. ✅ **Load** : Exporter en Parquet + charger dans DuckDB
4. ✅ **Quality** : Logging JSON, tests pytest > 80%, pre-commit
5. ✅ **Structure** : Architecture propre, configuration centralisée

---

## 📝 Spécifications techniques

### 1. Extract (Extraction)

**Sources de données :**

1. **Fichiers CSV historiques** :
   - `data/raw/transactions_*.csv` : Historique des transactions
   - Colonnes : `transaction_id`, `user_id`, `amount`, `currency`, `timestamp`, `status`

2. **API REST** :
   - URL : JSONPlaceholder (`https://jsonplaceholder.typicode.com/users`) ou créer un mock
   - Récupérer les informations utilisateurs : `id`, `name`, `email`, `city`

**Exigences :**
- Gestion d'erreurs robuste (try/except)
- Retry automatique en cas d'échec API (3 tentatives)
- Timeout sur les appels API (5 secondes)
- Logging de toutes les opérations

### 2. Transform (Transformation)

**Validation avec Pydantic :**

Créer des modèles Pydantic pour chaque source :

```python
class Transaction(BaseModel):
    transaction_id: str
    user_id: int
    amount: Decimal
    currency: str = Field(pattern="^[A-Z]{3}$")  # ISO 4217
    timestamp: datetime
    status: Literal["pending", "completed", "failed"]
    
    @field_validator('amount')
    def amount_positive(cls, v):
        if v <= 0:
            raise ValueError('Amount must be positive')
        return v

class User(BaseModel):
    id: int
    name: str
    email: EmailStr
    city: str
```

**Nettoyage avec Pandas :**
- Supprimer les doublons
- Gérer les valeurs manquantes
- Convertir les types de données
- Normaliser les formats (dates, devises)

**Enrichissement :**
- Joindre transactions et users sur `user_id`
- Calculer des métriques agrégées (total par utilisateur, par jour)
- Ajouter des flags de qualité (is_valid, has_complete_info)

### 3. Load (Chargement)

**Export Parquet :**
- Partitionner par date (`partition_cols=['year', 'month', 'day']`)
- Compression snappy
- Structure : `data/processed/transactions/year=2024/month=01/day=15/data.parquet`

**Chargement DuckDB :**
- Créer une base DuckDB locale
- Créer les tables `transactions`, `users`, `transactions_enriched`
- Charger les données depuis les Parquet
- Vérifier l'intégrité (count, checksums)

### 4. Qualité et tests

**Logging structuré (JSON) :**

```python
import structlog

logger = structlog.get_logger()
logger.info("extraction_started", source="csv", file_count=5)
logger.error("validation_failed", error=str(e), record_id=tx_id)
```

**Tests pytest :**
- Tests unitaires pour chaque fonction
- Tests d'intégration pour le pipeline complet
- Fixtures avec données de test
- Coverage > 80%

**Configuration pre-commit :**
- ruff (linter + formatter)
- mypy (type checker)
- pytest (tests)

**Pydantic Settings pour configuration :**

```python
class Settings(BaseSettings):
    api_url: str
    api_timeout: int = 5
    api_retries: int = 3
    db_path: str = "data/warehouse.duckdb"
    log_level: str = "INFO"
    
    class Config:
        env_file = ".env"
```

### 5. Structure du projet

```
pipeline/
├── src/
│   ├── __init__.py
│   ├── extract.py           # Extraction CSV + API
│   ├── transform.py         # Validation + transformation
│   ├── load.py              # Export Parquet + DuckDB
│   ├── models.py            # Modèles Pydantic
│   ├── config.py            # Configuration (Pydantic Settings)
│   ├── exceptions.py        # Exceptions custom
│   └── main.py              # Point d'entrée
├── tests/
│   ├── __init__.py
│   ├── test_extract.py
│   ├── test_transform.py
│   ├── test_load.py
│   └── conftest.py          # Fixtures pytest
├── data/
│   ├── raw/                 # CSV bruts
│   ├── processed/           # Parquet
│   └── warehouse.duckdb     # Base DuckDB
├── .env.example             # Exemple de configuration
├── .pre-commit-config.yaml
├── pyproject.toml
├── requirements.txt
└── README.md
```

---

## 📦 Livrables

### Checklist

- [ ] Structure de projet respectée
- [ ] `src/extract.py` : Lecture CSV + appel API avec retry
- [ ] `src/models.py` : Modèles Pydantic avec validation
- [ ] `src/transform.py` : Validation + nettoyage + enrichissement
- [ ] `src/load.py` : Export Parquet partitionné + chargement DuckDB
- [ ] `src/config.py` : Pydantic Settings
- [ ] `src/exceptions.py` : Exceptions custom
- [ ] `src/main.py` : Pipeline complet fonctionnel
- [ ] `tests/` : 8+ tests avec coverage > 80%
- [ ] `pyproject.toml` : Config ruff + pytest
- [ ] `.pre-commit-config.yaml` : Hooks Git
- [ ] `.env.example` : Variables d'environnement
- [ ] Logging structuré (JSON format)
- [ ] Type hints partout
- [ ] Docstrings (Google style)
- [ ] README.md complet

---

## 🎯 Grille d'évaluation

| Compétence | Critères d'évaluation | Points |
|------------|----------------------|--------|
| **Extract** | CSV + API, retry, gestion erreurs, logging | **/15** |
| **Validation Pydantic** | Modèles complets, validators, types corrects | **/15** |
| **Transform Pandas** | Nettoyage, enrichissement, jointures | **/15** |
| **Load** | Parquet partitionné + DuckDB + vérifications | **/15** |
| **Tests pytest** | 8+ tests, coverage > 80%, fixtures | **/15** |
| **Logging structuré** | JSON format, niveaux appropriés, contexte | **/10** |
| **Configuration** | pre-commit + ruff + Pydantic Settings | **/10** |
| **Qualité code** | Structure, type hints, docstrings, PEP 8 | **/5** |
| **TOTAL** | | **/100** |

### Critères de réussite

- ✅ **Acquis (≥ 70/100)** : Pipeline complet et professionnel
- 🚧 **En cours d'acquisition (50-69/100)** : Pipeline partiel
- ❌ **Non acquis (< 50/100)** : Pipeline incomplet ou non fonctionnel

---

## 🛠️ Préparation : Créer des données de test

In [ ]:
# Créer la structure de dossiers
import os

os.makedirs("pipeline/src", exist_ok=True)
os.makedirs("pipeline/tests", exist_ok=True)
os.makedirs("pipeline/data/raw", exist_ok=True)
os.makedirs("pipeline/data/processed", exist_ok=True)

print("✅ Structure de dossiers créée")

In [ ]:
%%writefile pipeline/data/raw/transactions_20240115.csv
transaction_id,user_id,amount,currency,timestamp,status
TXN001,1,150.50,EUR,2024-01-15 10:30:00,completed
TXN002,2,89.99,USD,2024-01-15 11:45:00,completed
TXN003,1,200.00,EUR,2024-01-15 14:20:00,pending
TXN004,3,45.75,GBP,2024-01-15 16:00:00,completed
TXN005,2,  ,USD,2024-01-15 17:30:00,failed
TXN006,4,300.00,EUR,2024-01-15 18:15:00,completed

In [ ]:
%%writefile pipeline/data/raw/transactions_20240116.csv
transaction_id,user_id,amount,currency,timestamp,status
TXN007,1,75.25,EUR,2024-01-16 09:00:00,completed
TXN008,3,120.00,GBP,2024-01-16 10:30:00,completed
TXN009,2,50.50,USD,2024-01-16 12:00:00,pending
TXN010,4,180.00,EUR,2024-01-16 15:45:00,completed

---

## 💡 Template : src/models.py

In [ ]:
%%writefile pipeline/src/models.py
"""Modèles Pydantic pour la validation des données."""

from datetime import datetime
from decimal import Decimal
from typing import Literal

from pydantic import BaseModel, EmailStr, Field, field_validator


class Transaction(BaseModel):
    """Modèle de transaction financière."""
    
    transaction_id: str = Field(..., min_length=5)
    user_id: int = Field(..., gt=0)
    amount: Decimal = Field(..., gt=0)
    currency: str = Field(..., pattern=r"^[A-Z]{3}$")
    timestamp: datetime
    status: Literal["pending", "completed", "failed"]
    
    @field_validator('amount')
    @classmethod
    def amount_positive(cls, v: Decimal) -> Decimal:
        """Valide que le montant est positif."""
        if v <= 0:
            raise ValueError("Amount must be positive")
        return v
    
    class Config:
        frozen = True  # Immutable


class User(BaseModel):
    """Modèle d'utilisateur."""
    
    id: int = Field(..., gt=0)
    name: str = Field(..., min_length=1)
    email: EmailStr
    city: str = Field(..., min_length=1)
    
    class Config:
        frozen = True


class TransactionEnriched(BaseModel):
    """Transaction enrichie avec informations utilisateur."""
    
    transaction_id: str
    user_id: int
    user_name: str
    user_email: str
    user_city: str
    amount: Decimal
    currency: str
    timestamp: datetime
    status: str
    year: int
    month: int
    day: int
    
    # TODO: Ajouter des computed fields si nécessaire

---

## 💡 Template : src/config.py

In [ ]:
%%writefile pipeline/src/config.py
"""Configuration centralisée avec Pydantic Settings."""

from pydantic_settings import BaseSettings, SettingsConfigDict


class Settings(BaseSettings):
    """Configuration de l'application."""
    
    # API
    api_url: str = "https://jsonplaceholder.typicode.com/users"
    api_timeout: int = 5
    api_retries: int = 3
    
    # Paths
    raw_data_path: str = "data/raw"
    processed_data_path: str = "data/processed"
    db_path: str = "data/warehouse.duckdb"
    
    # Logging
    log_level: str = "INFO"
    log_format: str = "json"
    
    # Processing
    batch_size: int = 1000
    partition_by: list[str] = ["year", "month", "day"]
    
    model_config = SettingsConfigDict(
        env_file=".env",
        env_file_encoding="utf-8",
        case_sensitive=False,
    )


# Instance globale
settings = Settings()

---

## 💡 Template : src/exceptions.py

In [ ]:
%%writefile pipeline/src/exceptions.py
"""Exceptions personnalisées pour le pipeline."""


class PipelineError(Exception):
    """Exception de base pour le pipeline."""
    pass


class ExtractionError(PipelineError):
    """Levée lors d'une erreur d'extraction."""
    pass


class ValidationError(PipelineError):
    """Levée lors d'une erreur de validation."""
    pass


class TransformationError(PipelineError):
    """Levée lors d'une erreur de transformation."""
    pass


class LoadError(PipelineError):
    """Levée lors d'une erreur de chargement."""
    pass

---

## 💡 Template : src/extract.py

In [ ]:
%%writefile pipeline/src/extract.py
"""Module d'extraction des données."""

import time
from pathlib import Path
from typing import Any

import pandas as pd
import requests
import structlog

from .config import settings
from .exceptions import ExtractionError

logger = structlog.get_logger()


def extract_csv_files(data_path: str) -> pd.DataFrame:
    """Extrait et concatène tous les fichiers CSV du dossier.
    
    Args:
        data_path: Chemin vers le dossier contenant les CSV.
        
    Returns:
        DataFrame Pandas avec toutes les données.
        
    Raises:
        ExtractionError: Si aucun fichier trouvé ou erreur de lecture.
    """
    logger.info("csv_extraction_started", path=data_path)
    
    # TODO: Implémenter
    # 1. Lister tous les fichiers CSV
    # 2. Lire chaque fichier avec pd.read_csv
    # 3. Concaténer tous les DataFrames
    # 4. Gérer les erreurs (fichier vide, format invalide, etc.)
    
    pass


def extract_api_data(url: str, timeout: int = 5, retries: int = 3) -> list[dict[str, Any]]:
    """Extrait des données depuis une API REST avec retry.
    
    Args:
        url: URL de l'API.
        timeout: Timeout en secondes.
        retries: Nombre de tentatives.
        
    Returns:
        Liste de dictionnaires avec les données.
        
    Raises:
        ExtractionError: Si toutes les tentatives échouent.
    """
    logger.info("api_extraction_started", url=url, retries=retries)
    
    # TODO: Implémenter avec retry
    # 1. Boucle sur le nombre de retries
    # 2. requests.get() avec timeout
    # 3. Vérifier response.status_code
    # 4. Parser le JSON
    # 5. En cas d'erreur, attendre avant retry (exponential backoff)
    # 6. Logger chaque tentative
    
    pass


def extract_all() -> tuple[pd.DataFrame, list[dict[str, Any]]]:
    """Extrait toutes les données (CSV + API).
    
    Returns:
        Tuple (transactions_df, users_list).
    """
    transactions_df = extract_csv_files(settings.raw_data_path)
    users_list = extract_api_data(settings.api_url, settings.api_timeout, settings.api_retries)
    
    logger.info(
        "extraction_completed",
        transactions_count=len(transactions_df),
        users_count=len(users_list),
    )
    
    return transactions_df, users_list

---

## 💡 Template : src/transform.py

In [ ]:
%%writefile pipeline/src/transform.py
"""Module de transformation des données."""

from typing import Any

import pandas as pd
import structlog
from pydantic import ValidationError

from .exceptions import TransformationError, ValidationError as CustomValidationError
from .models import Transaction, TransactionEnriched, User

logger = structlog.get_logger()


def validate_transactions(df: pd.DataFrame) -> list[Transaction]:
    """Valide les transactions avec Pydantic.
    
    Args:
        df: DataFrame avec les transactions brutes.
        
    Returns:
        Liste de transactions validées.
    """
    logger.info("validation_started", entity="transactions", count=len(df))
    
    valid_transactions = []
    errors = []
    
    # TODO: Implémenter
    # 1. Itérer sur les lignes du DataFrame
    # 2. Pour chaque ligne, essayer de créer un Transaction
    # 3. Si ValidationError, logger l'erreur et continuer
    # 4. Si OK, ajouter à valid_transactions
    # 5. Logger le nombre d'erreurs
    
    pass


def validate_users(users_list: list[dict[str, Any]]) -> list[User]:
    """Valide les utilisateurs avec Pydantic.
    
    Args:
        users_list: Liste de dictionnaires avec les users.
        
    Returns:
        Liste d'utilisateurs validés.
    """
    logger.info("validation_started", entity="users", count=len(users_list))
    
    # TODO: Implémenter (même logique que validate_transactions)
    
    pass


def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Nettoie le DataFrame.
    
    Args:
        df: DataFrame brut.
        
    Returns:
        DataFrame nettoyé.
    """
    logger.info("cleaning_started", rows_before=len(df))
    
    # TODO: Implémenter
    # 1. Supprimer les doublons (drop_duplicates)
    # 2. Gérer les valeurs manquantes
    # 3. Convertir les types
    # 4. Normaliser les formats
    
    pass


def enrich_transactions(
    transactions: list[Transaction], users: list[User]
) -> pd.DataFrame:
    """Enrichit les transactions avec les informations utilisateurs.
    
    Args:
        transactions: Liste de transactions validées.
        users: Liste d'utilisateurs validés.
        
    Returns:
        DataFrame enrichi.
    """
    logger.info("enrichment_started")
    
    # TODO: Implémenter
    # 1. Convertir les listes Pydantic en DataFrames
    # 2. Joindre sur user_id
    # 3. Ajouter des colonnes dérivées (year, month, day)
    # 4. Retourner le DataFrame enrichi
    
    pass


def transform_all(
    transactions_df: pd.DataFrame, users_list: list[dict[str, Any]]
) -> pd.DataFrame:
    """Pipeline complet de transformation.
    
    Args:
        transactions_df: DataFrame brut des transactions.
        users_list: Liste brute des utilisateurs.
        
    Returns:
        DataFrame enrichi et validé.
    """
    # Validation
    valid_transactions = validate_transactions(transactions_df)
    valid_users = validate_users(users_list)
    
    # Enrichissement
    enriched_df = enrich_transactions(valid_transactions, valid_users)
    
    # Nettoyage final
    clean_df = clean_dataframe(enriched_df)
    
    logger.info("transformation_completed", rows=len(clean_df))
    
    return clean_df

---

## 💡 Template : src/load.py

In [ ]:
%%writefile pipeline/src/load.py
"""Module de chargement des données."""

import duckdb
import pandas as pd
import structlog

from .config import settings
from .exceptions import LoadError

logger = structlog.get_logger()


def export_to_parquet(df: pd.DataFrame, output_path: str, partition_cols: list[str]) -> None:
    """Exporte le DataFrame en Parquet partitionné.
    
    Args:
        df: DataFrame à exporter.
        output_path: Chemin de base pour l'export.
        partition_cols: Colonnes de partitionnement.
    """
    logger.info("parquet_export_started", path=output_path, partitions=partition_cols)
    
    # TODO: Implémenter
    # 1. Utiliser df.to_parquet() avec partition_cols
    # 2. Spécifier compression='snappy'
    # 3. Gérer les erreurs (permissions, espace disque)
    
    pass


def load_to_duckdb(df: pd.DataFrame, db_path: str, table_name: str) -> None:
    """Charge le DataFrame dans DuckDB.
    
    Args:
        df: DataFrame à charger.
        db_path: Chemin vers la base DuckDB.
        table_name: Nom de la table.
    """
    logger.info("duckdb_load_started", table=table_name, rows=len(df))
    
    # TODO: Implémenter
    # 1. Se connecter à DuckDB (duckdb.connect)
    # 2. Créer la table si elle n'existe pas
    # 3. Insérer les données (depuis DataFrame ou Parquet)
    # 4. Vérifier l'intégrité (count)
    # 5. Fermer la connexion
    
    pass


def verify_data_integrity(db_path: str, table_name: str, expected_count: int) -> bool:
    """Vérifie l'intégrité des données chargées.
    
    Args:
        db_path: Chemin vers la base DuckDB.
        table_name: Nom de la table.
        expected_count: Nombre de lignes attendu.
        
    Returns:
        True si intégrité OK.
    """
    # TODO: Implémenter
    # 1. Compter les lignes dans la table
    # 2. Comparer avec expected_count
    # 3. Logger le résultat
    
    pass


def load_all(df: pd.DataFrame) -> None:
    """Pipeline complet de chargement.
    
    Args:
        df: DataFrame enrichi à charger.
    """
    # Export Parquet
    export_to_parquet(
        df, settings.processed_data_path, settings.partition_by
    )
    
    # Load DuckDB
    load_to_duckdb(df, settings.db_path, "transactions_enriched")
    
    # Vérification
    is_valid = verify_data_integrity(settings.db_path, "transactions_enriched", len(df))
    
    if not is_valid:
        raise LoadError("Data integrity check failed")
    
    logger.info("load_completed")

---

## 💡 Template : src/main.py

In [ ]:
%%writefile pipeline/src/main.py
"""Point d'entrée du pipeline ETL."""

import structlog

from .config import settings
from .extract import extract_all
from .load import load_all
from .transform import transform_all

# Configuration du logging
structlog.configure(
    processors=[
        structlog.processors.TimeStamper(fmt="iso"),
        structlog.processors.add_log_level,
        structlog.processors.JSONRenderer(),
    ]
)

logger = structlog.get_logger()


def run_pipeline() -> None:
    """Exécute le pipeline ETL complet."""
    logger.info("pipeline_started")
    
    try:
        # Extract
        transactions_df, users_list = extract_all()
        
        # Transform
        enriched_df = transform_all(transactions_df, users_list)
        
        # Load
        load_all(enriched_df)
        
        logger.info("pipeline_completed")
        
    except Exception as e:
        logger.error("pipeline_failed", error=str(e), error_type=type(e).__name__)
        raise


if __name__ == "__main__":
    run_pipeline()

---

## 💡 Template : tests/conftest.py

In [ ]:
%%writefile pipeline/tests/conftest.py
"""Fixtures pytest partagées."""

from datetime import datetime
from decimal import Decimal

import pandas as pd
import pytest


@pytest.fixture
def sample_transactions_df():
    """Fixture avec des transactions de test."""
    return pd.DataFrame([
        {
            "transaction_id": "TXN001",
            "user_id": 1,
            "amount": 150.50,
            "currency": "EUR",
            "timestamp": "2024-01-15 10:30:00",
            "status": "completed",
        },
        {
            "transaction_id": "TXN002",
            "user_id": 2,
            "amount": 89.99,
            "currency": "USD",
            "timestamp": "2024-01-15 11:45:00",
            "status": "completed",
        },
    ])


@pytest.fixture
def sample_users_list():
    """Fixture avec des utilisateurs de test."""
    return [
        {"id": 1, "name": "Alice", "email": "alice@example.com", "city": "Paris"},
        {"id": 2, "name": "Bob", "email": "bob@example.com", "city": "Lyon"},
    ]


# TODO: Ajouter d'autres fixtures utiles

---

## 💡 Template : pyproject.toml et .env.example

In [ ]:
%%writefile pipeline/pyproject.toml
[project]
name = "pipeline-etl"
version = "1.0.0"
requires-python = ">=3.11"
dependencies = [
    "pandas>=2.0.0",
    "pydantic>=2.0.0",
    "pydantic-settings>=2.0.0",
    "requests>=2.31.0",
    "duckdb>=0.9.0",
    "structlog>=23.0.0",
    "pytest>=7.4.0",
    "pytest-cov>=4.1.0",
]

[tool.ruff]
line-length = 100
target-version = "py311"

[tool.ruff.lint]
select = ["E", "W", "F", "I", "B", "C4", "UP"]

[tool.pytest.ini_options]
testpaths = ["tests"]
addopts = "-v --cov=src --cov-report=term-missing --cov-report=html"

[tool.mypy]
python_version = "3.11"
warn_return_any = true
disallow_untyped_defs = true

In [ ]:
%%writefile pipeline/.env.example
# Configuration du pipeline ETL

# API
API_URL=https://jsonplaceholder.typicode.com/users
API_TIMEOUT=5
API_RETRIES=3

# Paths
RAW_DATA_PATH=data/raw
PROCESSED_DATA_PATH=data/processed
DB_PATH=data/warehouse.duckdb

# Logging
LOG_LEVEL=INFO
LOG_FORMAT=json

---

## 🚀 Étapes recommandées

1. **Créer la structure** : Dossiers et fichiers vides
2. **Implémenter Extract** : CSV puis API (avec retry)
3. **Implémenter les modèles Pydantic** : Transaction et User
4. **Implémenter Transform** : Validation puis enrichissement
5. **Implémenter Load** : Parquet puis DuckDB
6. **Tester chaque module** : Un test à la fois
7. **Intégrer le pipeline** : main.py
8. **Ajouter le logging** : structlog partout
9. **Configurer les hooks** : pre-commit
10. **Documentation** : README.md

---

## 📚 Ressources

### Documentation
- [Pydantic](https://docs.pydantic.dev/)
- [Pandas](https://pandas.pydata.org/docs/)
- [DuckDB](https://duckdb.org/docs/)
- [structlog](https://www.structlog.org/)
- [pytest](https://docs.pytest.org/)
- [Parquet](https://arrow.apache.org/docs/python/parquet.html)

### Concepts ETL
- Extract : Ingestion de données multi-sources
- Transform : Validation, nettoyage, enrichissement
- Load : Persistence optimisée (Parquet, DuckDB)
- Data quality : Validation Pydantic, vérifications
- Observability : Logging structuré, métriques

---

## ✅ Critères de validation finale

Avant de soumettre, vérifiez que :

1. ✅ `python -m src.main` s'exécute sans erreur
2. ✅ Les fichiers Parquet sont générés et partitionnés
3. ✅ La base DuckDB contient les données
4. ✅ `pytest` : tous les tests passent, coverage > 80%
5. ✅ `ruff check .` : aucune erreur
6. ✅ `mypy src/` : aucune erreur
7. ✅ Les logs sont en JSON et structurés
8. ✅ Les modèles Pydantic valident correctement
9. ✅ Le retry API fonctionne
10. ✅ Les exceptions custom sont utilisées
11. ✅ La configuration .env fonctionne
12. ✅ Le README explique l'installation et l'utilisation

**Bon courage pour ce projet expert ! 🚀**